In [130]:
from sklearn.linear_model import LinearRegression 
import pandas as pd
import numpy as np
import pickle
from lightgbm import LGBMRegressor

### Инициализируем реальные данные из validate и заодно инициализируем их среднее по клипам

In [131]:
colls = ['file_path', 'valence', 'arousal']
drop_colls = []
for i in range(137):
    colls.append('mark_{}'.format(i))
    drop_colls.append('mark_{}'.format(i))
actual_data = pd.read_table('valid_data_with_landmarks.txt', names = colls, delim_whitespace=True)
actual_data.drop(columns = drop_colls, inplace = True)

In [132]:
def get_clip_unix(path):
    file = path.split('/')[1:]
    file = '/'.join(file)
    return '\\'+file[:file.find('.')]

In [133]:
def get_clip(path):
    file = path.split('\\')[:-1]
    file = '/'.join(file)
    return '\\'+file[:file.find('.')]

actual_data['file_path'] = actual_data['file_path'].apply(get_clip)

In [134]:
def means(df):
    return df.groupby('file_path', sort = False).mean().reset_index(level=0)

In [135]:
actual_data_mean = means(actual_data)

## Считаем модели Ивана

#### Аудио_1

In [136]:
audio_test_I = pd.read_table('ready_models/results/AEmotion_Ivan_result_test.txt', delim_whitespace=True, names = ['file_path', 'valence_I1', 'arousal_I1'])
audio_I = pd.read_table('ready_models/results/AEmotion_Ivan_result_valid.txt', delim_whitespace=True, names = ['file_path', 'valence_I1', 'arousal_I1'])

#### Аудио_2

In [137]:
audio_test_I2 = pd.read_table('ready_models/results/AEmotion_Ivan_result_test_3.txt', delim_whitespace=True, names = ['file_path', 'valence_I2', 'arousal_I2'])
audio_I2 = pd.read_table('ready_models/results/AEmotion_Ivan_result_valid_3.txt', delim_whitespace=True, names = ['file_path', 'valence_I2', 'arousal_I2'])

## Модель Михаила

#### Аудио_1

In [138]:
audio_test_M = pd.read_table('ready_models/results/AEmotion_test_result.txt', delim_whitespace=True, names = ['file_path', 'valence_M1', 'arousal_M1'])
audio_M = pd.read_table('ready_models/results/AEmotion_valid_result.txt', delim_whitespace=True, names = ['file_path', 'valence_M1', 'arousal_M1'])

audio_test_M['file_path'] = audio_test_M['file_path'].apply(get_clip_unix)

audio_M['file_path'] = audio_M['file_path'].apply(get_clip_unix)

### Видео_1

In [139]:
video_test_M = pd.read_table('ready_models/results/VEmotion_test_result.txt', delim_whitespace=True, names = ['file_path', 'valence_M2', 'arousal_M2'])
video_M = pd.read_table('ready_models/results/VEmotion_valid_result.txt', delim_whitespace=True, names = ['file_path', 'valence_M2', 'arousal_M2'])

video_test_M['file_path'] = video_test_M['file_path'].apply(get_clip_unix)

video_M['file_path'] = video_M['file_path'].apply(get_clip_unix)

In [140]:
video_M = means(video_M)
video_test_M = means(video_test_M)

## Попробуем выгрузить активации

### Аудио 1 Михаил

In [141]:
f = open('ready_models/results/activations/audio_activations_test.pkl', 'rb')
audio_feat_test = pickle.load(f)
f.close()

audio_feat_test = audio_feat_test.reshape((255, 1024))

audio_feat_test = pd.DataFrame(audio_feat_test, columns = ['feat_{}'.format(i) for i in range(1024)])

audio_test_M = pd.concat([audio_test_M, audio_feat_test], axis = 1)

In [142]:
f = open('ready_models/results/activations/audio_activations_valid.pkl', 'rb')
audio_feat_valid = pickle.load(f)
f.close()

audio_feat_valid = audio_feat_valid.reshape((226, 1024))

audio_feat_valid = pd.DataFrame(audio_feat_valid, columns = ['feat_{}'.format(i) for i in range(1024)])

audio_M = pd.concat([audio_M, audio_feat_valid], axis = 1)

### Видео 1 Михаил

In [143]:
def PCA(df):
#     df = video_feat_test.loc[video_feat_test.file_path == '\\664ea0ccc_25/utterance_576']
    return df.values[:,1:].flatten()

In [144]:
f = open('ready_models/results/activations/video_activations_test.pkl', 'rb')
video_feat_test = pickle.load(f)
f.close()

video_feat_test = video_feat_test.reshape(video_feat_test.shape[:2])
video_feat_test = pd.DataFrame(video_feat_test, columns = ['feat_{}'.format(i) for i in range(video_feat_test.shape[1])])
video_feat_test = pd.concat([actual_data.file_path, video_feat_test], axis = 1)

features_df = video_feat_test.groupby('file_path').apply(PCA)
features_df = pd.DataFrame({'feat' : features_df}).reset_index()

In [145]:
PCA_test = []
for val in features_df.values[1:]:
    PCA_test.append(np.array(val[1:][0], dtype = 'float32').tolist())

In [146]:
f = open('ready_models/results/activations/video_activations_valid.pkl', 'rb')
video_feat_valid = pickle.load(f)
f.close()

video_feat_valid = video_feat_valid.reshape(video_feat_valid.shape[:2])
video_feat_valid = pd.DataFrame(video_feat_valid, columns = ['feat_{}'.format(i) for i in range(video_feat_valid.shape[1])])
video_feat_valid = pd.concat([actual_data.file_path, video_feat_valid], axis = 1)

features_df = video_feat_valid.groupby('file_path').apply(PCA)
features_df = pd.DataFrame({'feat' : features_df}).reset_index()

In [147]:
PCA_train = []
for val in features_df.values:
    PCA_train.append(np.array(val[1:][0], dtype = 'float32').tolist())

In [148]:
def numpy_fillna(data):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(lens.max()) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out

In [149]:
PCA_train = numpy_fillna(np.array(PCA_train))

In [ ]:
from sklearn.cross_decomposition import CCA
PCA = CCA(n_components=100)
PCA_train = PCA.fit_transform(X = PCA_train, y=actual_data_mean[['valence', 'arousal']])

In [ ]:
PCA_df = pd.DataFrame(PCA_train, columns = ['PCA_feat_{}'.format(i) for i in range(226)])

In [ ]:
PCA_test = PCA.transform(PCA_test)

## Сделаем train test

In [108]:
X_train = audio_I2.merge(video_M, on = 'file_path')
# X_train = X_train.merge(audio_I2, on = 'file_path')
X_train.head()

,file_path,valence_I2,arousal_I2,valence_M2,arousal_M2
0,\664ea0ccc_25/utterance_576,0.381382,0.100120,0.268021,0.148986
1,\bee9cd4cf_2/utterance_42,0.356596,0.272787,0.612218,0.434692
2,\664ea0ccc_31/utterance_718,0.407344,0.138530,0.458484,0.171215
3,\74de88564/utterance_7,0.386217,0.182176,0.085194,0.887938
4,\664ea0ccc_25/utterance_569,0.389408,0.151607,0.326090,0.167808


In [109]:
X_test = audio_test_I2.merge(video_test_M, on = 'file_path')
# X_test = X_test.merge(audio_test_I2, on = 'file_path')
X_test.head()

,file_path,valence_I2,arousal_I2,valence_M2,arousal_M2
0,\664ea0ccc_25/utterance_580,0.388769,0.104807,0.273778,0.039738
1,\664ea0ccc_25/utterance_560,0.380622,0.098278,0.294232,0.094626
2,\664ea0ccc_14/utterance_322,0.402219,0.109847,0.338395,0.152145
3,\74de88564/utterance_2,0.340513,0.358777,0.079900,0.891378
4,\e3b57cfd8_7/utterance_117,0.387916,0.106560,0.738702,-0.125857


In [110]:
from sklearn.model_selection import train_test_split

In [111]:
valence_indx = list(range(1,len(X_train.columns),2))
arous_indx = list(range(2,len(X_train.columns),2))

XV_train = X_train.drop(columns = ['file_path', 'arousal_M2', 'arousal_I2'])
yV_train = actual_data_mean['valence']

XA_train = X_train.drop(columns = ['file_path', 'valence_M2', 'valence_I2'])
yA_train = actual_data_mean['arousal']

In [112]:
XV_test = X_test.drop(columns = ['file_path', 'arousal_M2', 'arousal_I2'])
XA_test = X_test.drop(columns = ['file_path', 'valence_M2', 'valence_I2'])

In [113]:
# XV_train, XV_val, yV_train, yV_val = train_test_split(XV_train, yV_train, test_size=0.33, random_state=42)
# XA_train, XA_val, yA_train, yA_val = train_test_split(XA_train, yA_train, test_size=0.33, random_state=42)

# XA_train.reset_index(inplace = True, drop = True)
# XA_val.reset_index(inplace = True, drop = True)
# yA_train.reset_index(inplace = True, drop = True)
# yA_val.reset_index(inplace = True, drop = True)
# XV_train.reset_index(inplace = True, drop = True)
# XV_val.reset_index(inplace = True, drop = True)
# yV_train.reset_index(inplace = True, drop = True)
# yV_val.reset_index(inplace = True, drop = True)

## Посчитаем ошибки

In [114]:
def error(actual, predict, class_num):
    pred = predict[predict.columns[class_num]].values
    act = actual[actual.columns[class_num%2]].values
    all = len(act)
    pos = 0
    for i in range(len(act)):
        a = act[i]
        p = pred[i]
        if abs(a - p) < 0.1:
            pos += 1
    return pos/all

### Обучим линейную регрессию

In [115]:
LR_A = LinearRegression(normalize=False)
LR_A.fit(XA_train, yA_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [116]:
LR_V = LinearRegression(normalize=False)
LR_V.fit(XV_train, yV_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### Обучим LightGBM

In [117]:
def error(actual, predict, class_num):
    pred = predict[predict.columns[class_num]].values
    act = actual[actual.columns[class_num]].values
    all = len(act)
    pos = 0
    for i in range(len(act)):
        a = act[i]
        p = pred[i]
        if abs(a - p) < 0.1:
            pos += 1
    return pos/all

def print_acc_val(act, pred):
    val_err = error(act, pred, 0)
    print('valence accuracy: ', val_err)

    ar_err = error(act, pred, 1)
    print('arousal accuracy: ', ar_err)

In [118]:
def test(XV, XA, yV, yA):
    tree_A = LGBMRegressor(n_estimators=ESTIMATORS_A, random_state=1, learning_rate=LEARRATE_A, depth = DEPTH_V)
    tree_A.fit(XA_train, yA_train)

    tree_V = LGBMRegressor(n_estimators=ESTIMATORS_V, random_state=1, learning_rate=LEARRATE_V, depth = DEPTH_V)
    tree_V.fit(XV_train, yV_train)

    type = 'GBM'

    if type == 'LR':
        valence_pred = LR_V.predict(XV)
        arousal_pred = LR_A.predict(XA)
    else:
        valence_pred = tree_V.predict(XV)
        arousal_pred = tree_A.predict(XA)

    pred_data = pd.concat([pd.DataFrame(valence_pred, columns = ['valence']),\
                           pd.DataFrame(arousal_pred, columns = ['arousal'])], axis = 1) 

    act_data = pd.concat([yV, yA], axis = 1)
    return act_data, pred_data

In [119]:
ESTIMATORS_V = 1000
LEARRATE_V = 0.0001
DEPTH_V = -1

ESTIMATORS_A = 1000
LEARRATE_A = 0.05
DEPTH_A = -1

if type == 'LR':
    print('LR')
else:
    print('tree')
    
# print('val')
# act_data, pred_data = test(XV_val, XA_val, yV_val, yA_val)
# print_acc_val(act_data, pred_data)
# print('train')
# act_data, pred_data = test(XV_train, XA_train, yV_train, yA_train)
# print_acc_val(act_data, pred_data)

tree


In [120]:
tree_A = LGBMRegressor(n_estimators=ESTIMATORS_A, random_state=1, learning_rate=LEARRATE_A, depth = DEPTH_V)
tree_A.fit(XA_train, yA_train)

tree_V = LGBMRegressor(n_estimators=ESTIMATORS_V, random_state=1, learning_rate=LEARRATE_V, depth = DEPTH_V)
tree_V.fit(XV_train, yV_train)

# tree_V.booster_.save_model('valence_32.txt')

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       depth=-1, learning_rate=0.0001, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
       n_jobs=-1, num_leaves=31, objective=None, random_state=1,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

## Модели Михаила

### аудио 1

In [121]:
def print_acc(act, pred):
    new_df = act.merge(pred, on = 'file_path')
    val_err = error(new_df[new_df.columns[1:3]], new_df[new_df.columns[3:5]], 0)
    print('valence accuracy: ', val_err)

    ar_err = error(new_df[new_df.columns[1:3]], new_df[new_df.columns[3:5]], 1)
    print('arousal accuracy: ', ar_err)

In [122]:
print_acc(actual_data_mean, audio_M)

valence accuracy:  0.2345132743362832
arousal accuracy:  0.2168141592920354


### Видео 1

In [123]:
print_acc(actual_data_mean, video_M)

valence accuracy:  0.3495575221238938
arousal accuracy:  0.2610619469026549


### Модель Ивана аудио 1

In [124]:
print_acc(actual_data_mean, audio_I)

valence accuracy:  0.336283185840708
arousal accuracy:  0.2345132743362832


### Модель Ивана аудио 2

In [125]:
print_acc(actual_data_mean, audio_I2)

valence accuracy:  0.30973451327433627
arousal accuracy:  0.2831858407079646


# SUBMIT

In [126]:
import lightgbm as lgb

In [ ]:
tree_V = lgb.Booster(model_file='valence_32.txt')
tree_A = lgb.Booster(model_file='arous_28.txt')

In [127]:
if type == 'LR':
    valence_pred = LR_V.predict(XV_test)
    arousal_pred = LR_A.predict(XA_test)
else:
    valence_pred = tree_V.predict(XV_test)
    arousal_pred = tree_A.predict(XA_test)

pred_data = pd.concat([pd.DataFrame(valence_pred, columns = ['valence']),\
                       pd.DataFrame(arousal_pred, columns = ['arousal'])], axis = 1) 


In [128]:
pred_data = audio_test_I.copy()
pred_data.valence_I1 = valence_pred 
pred_data.arousal_I1 = arousal_pred 

In [129]:
pred_data.to_csv("prediction.txt", sep=" ", header=False, index=False)